# UNet with Resnet 50 Backbone

Train Dataset Size: 

Val Dataset Size: 

Test Dataset Size:

In [1]:
# %pip install torch -q
# %pip install opencv-python -q
# %pip install pycocotools -q
# %pip install timm==0.6.12 -q
# %pip install ipdb -q

In [2]:
from backbones_unet.model.unet import Unet
from backbones_unet.utils.dataset import SemanticSegmentationDataset
from backbones_unet.model.losses import DiceLoss
from backbones_unet.utils.trainer import Trainer
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from convert_coco_ann_to_mask import convert_coco_to_mask
from torchvision.datasets import ImageFolder

import torchvision
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

/home/sush/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [3]:
# Test Installation
random_tensor = torch.rand((1, 3, 64, 64))
model = Unet(in_channels=3, num_classes=1) # if no backbone specified, will default to Resnet50
print(model.predict(random_tensor))
# summary(model, random_tensor)

tensor([[[[ 2.6458,  3.0477,  1.9521,  ..., -0.9206, -1.2031,  0.3762],
          [ 2.4717,  4.4546,  1.0883,  ..., -2.4670, -1.3348,  1.2135],
          [ 0.3253,  0.2207, -0.5515,  ..., -0.8310,  1.0622,  1.1272],
          ...,
          [-0.4217, -0.5321,  0.5751,  ...,  0.5326,  0.6465,  0.6894],
          [-1.4970, -2.0495,  0.0494,  ..., -0.8868,  0.3673, -0.1797],
          [-0.5489, -1.8801, -0.4927,  ..., -0.0623,  0.8461,  1.2144]]]])


In [4]:
# Feel free to add more items here
config = {
    "lr"         : 1e-4,
    "epochs"     : 100,
    "batch_size" : 1,  # Increase if your device can handle it
    "num_classes": 1,
    'truncated_normal_mean' : 0,
    'truncated_normal_std' : 0.2,
}

In [5]:
# create a torch.utils.data.Dataset/DataLoader
annotation_json_path = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/annotation.json'
train_img_path = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/images'
train_mask_path = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/masks'

train_img_path_for_ImageFolder_dataloader = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/images_with_class/'

#! Temporarily using train and val images as same
val_img_path = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/images'
val_mask_path = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/masks'

test_img_path = '/home/sush/klab2/rosbags_collated/sensors_2023-08-03-15-19-03_0/images'

# img_size = (1384, 1032) # = width, height            # currently PtGrey images
img_size = (1024, 1024)

## Extract Masks from the COCO annotations (if not already done)

In [6]:
# convert_coco_to_mask(input_json=annotation_json_path, image_folder=train_img_path, output_folder=train_mask_path)

In [7]:
# Find mean and std of your dataset:
def get_mean_and_std_calculated(IMAGE_DATA_DIR):
    """
    NOTE: The ImageFolder dataloader requires the following file structure:

    root
    |
    └── cat (class label)
        |
        ├──img_2.png
        └──img_1.png

    """
    train_dataset = ImageFolder(IMAGE_DATA_DIR, transform=torchvision.transforms.ToTensor())

    # Initialize lists to store channel-wise means and standard deviations
    channel_wise_means = [0.0, 0.0, 0.0]
    channel_wise_stds = [0.0, 0.0, 0.0]

    # Iterate through the training dataset to calculate means and standard deviations
    for image, _ in train_dataset:
        for i in range(3):  # Assuming RGB images
            channel_wise_means[i] += image[i, :, :].mean().item()
            channel_wise_stds[i] += image[i, :, :].std().item()

    # Calculate the mean and standard deviation for each channel
    num_samples = len(train_dataset)
    channel_wise_means = [mean / num_samples for mean in channel_wise_means]
    channel_wise_stds = [std / num_samples for std in channel_wise_stds]

    # Print the mean and standard deviation for each channel
    print("Mean:", channel_wise_means)
    print("Std:", channel_wise_stds)

    return channel_wise_means, channel_wise_stds

# means, stds = get_mean_and_std_calculated(train_img_path_for_ImageFolder_dataloader)
means = [0.44895144719250346, 0.4951483853617493, 0.4498602793532975]
stds = [0.21388493326245522, 0.24571933703763144, 0.22413276759337405]

normalize_transform = torchvision.transforms.Compose([
    torchvision.transforms.Normalize(mean=means, std=stds) # always normalize only after tensor conversion
    ])

In [8]:
train_dataset = SemanticSegmentationDataset(img_paths=train_img_path, mask_paths=train_mask_path, size=img_size, mode='binary', normalize=normalize_transform, transformations=None)
val_dataset = SemanticSegmentationDataset(img_paths=val_img_path, mask_paths=val_mask_path, size=img_size, mode='binary', normalize=normalize_transform, transformations=None)
test_dataset = SemanticSegmentationDataset(img_paths=val_img_path, mask_paths=None, size=img_size, normalize=normalize_transform, transformations=None)

temp = train_dataset.__getitem__(1)

# Create data loaders
train_loader = DataLoader(
    dataset     = train_dataset,
    batch_size  = config['batch_size'],
    shuffle     = True,
    num_workers = 4,
    pin_memory  = True
)

val_loader = DataLoader(
    dataset     = val_dataset,
    batch_size  = config['batch_size'],
    shuffle     = False,
    num_workers = 2
)

test_loader = DataLoader(
    dataset     = test_dataset,
    batch_size  = config['batch_size'],
    shuffle     = False,
    drop_last   = False,
    num_workers = 2)

In [9]:
model = Unet(
    # backbone='convnext_base', # backbone network name
    backbone='resnet50',
    preprocessing=True,
    in_channels=3, # input channels (1 for gray-scale images, 3 for RGB, etc.)
    num_classes=config["num_classes"],  # output channels (number of classes in your dataset)
    encoder_freeze=True,
    pretrained=True,
)

# model = model().to(device)
random_tensor = torch.rand((1, 3, 1024, 1024))
print(model.predict(random_tensor))

tensor([[[[-1.0622,  1.4280, -0.6979,  ...,  2.4224,  2.4695,  0.3204],
          [ 2.1890,  2.9161,  3.5348,  ...,  1.3363,  2.0412, -0.1288],
          [ 2.3432,  1.6509,  1.6003,  ..., -2.1547, -0.5840,  2.6106],
          ...,
          [ 4.4475,  5.9496,  6.7059,  ...,  7.6961,  5.3557,  4.7004],
          [ 3.9954,  4.8844,  5.5944,  ...,  5.1450, -0.0916,  2.5199],
          [ 3.2162,  3.3522,  3.5893,  ...,  3.7011,  0.7092,  0.3289]]]])


In [10]:
# Define wandb credentials

import wandb
wandb.login(key="49efd84d0e342f343fb91401332234dea4a3ffe2") #API Key is in your wandb account, under settings (wandb.ai/settings)

run = wandb.init(
    name = "UNet_with_resnet_50", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "IDL_Project_Segmentation", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sushantj. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/sush/.netrc


In [11]:
checkpoint_path = '/home/sush/klab2/Segmentation_Models/checkpoints'

In [13]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=config['lr'], betas=(0.9, 0.999), weight_decay=0.05)
gamma = 0.8
milestones = [10,20,40,60,80]

# scheduler1 = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=0.9, total_iters=5)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=milestones, gamma=gamma)
# scheduler3 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
# scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[scheduler1, scheduler2, scheduler3], milestones=[20, 51])

mixed_precision_scaler = torch.cuda.amp.GradScaler()

trainer = Trainer(
    model=model,              # UNet model with Resnet50 backbone
    criterion=DiceLoss(),     # loss function
    optimizer=optimizer,
    epochs=10,
    # scaler=mixed_precision_scaler,
    lr_scheduler=scheduler,
    device=device,
    checkpoint_path=checkpoint_path
)

trainer.fit(train_loader, val_loader)

Traning Model on 10 epochs:  10%|█         | 1/10 [00:37<05:36, 37.36s/it]

Val Loss nan


Traning Model on 10 epochs:  20%|██        | 2/10 [01:17<05:10, 38.80s/it]

Val Loss nan


Traning Model on 10 epochs:  20%|██        | 2/10 [01:19<05:19, 39.92s/it]


KeyboardInterrupt: 